In [2]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import Tentativa_de_fazer_com_OO as mod


In [3]:
dados = pd.read_csv("df_filt.csv")
teste = pd.read_csv("df_test.csv")



In [ ]:
# Função para criar janelas deslizantes de 1s com 50% de overlap

def sliding_windows(df, window_s=1.0, overlap=0.5, time_col='Timestamp'):
    df = df.copy()
    if time_col not in df.columns:
        raise ValueError(f'Coluna de tempo {time_col} não encontrada no DataFrame')
    df[time_col] = pd.to_datetime(df[time_col])
    start = df[time_col].min()
    end = df[time_col].max()
    if pd.isna(start) or pd.isna(end):
        return []
    step = window_s * (1 - overlap)
    freq = pd.Timedelta(seconds=step)
    starts = pd.date_range(start=start, end=end, freq=freq)
    windows = []
    for s in starts:
        e = s + pd.Timedelta(seconds=window_s)
        win = df[(df[time_col] >= s) & (df[time_col] < e)]
        windows.append(win)
    return windows

# Exemplo de uso: separa o dataset em janelas
janelas = sliding_windows(dados, window_s=1.0, overlap=0.5)
print(f'Total de janelas criadas: {len(janelas)}')
# Exibe a primeira janela
janelas[0].head()

MemoryError: Unable to allocate 273. MiB for an array with shape (35769222,) and data type int64

In [4]:
# Cria janelas de 1 segundo com 50% de overlap e agrega features por janela
def sliding_windows(df, window_s=1.0, overlap=0.5, time_col='Timestamp'):
    df = df.copy()
    if time_col not in df.columns:
        raise ValueError(f'Coluna de tempo {time_col} não encontrada no DataFrame')
    df[time_col] = pd.to_datetime(df[time_col])

    start = df[time_col].min()
    end = df[time_col].max()
    if pd.isna(start) or pd.isna(end):
        return pd.DataFrame()

    step = window_s * (1 - overlap)
    freq = pd.Timedelta(seconds=step)
    starts = pd.date_range(start=start, end=end, freq=freq)

    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_candidates = ['Type', 'Position', 'Breed', 'Subject']
    categorical_cols = [c for c in categorical_candidates if c in df.columns]

    rows = []
    for s in starts:
        e = s + pd.Timedelta(seconds=window_s)
        win = df[(df[time_col] >= s) & (df[time_col] < e)]
        features = {'window_start': s, 'window_end': e, 'n_samples': len(win)}

        if len(win) == 0:
            for c in numeric_cols:
                features[f'mean_{c}'] = np.nan
                features[f'std_{c}'] = np.nan
            for c in categorical_cols:
                features[c] = None
        else:
            for c in numeric_cols:
                features[f'mean_{c}'] = win[c].mean()
                features[f'std_{c}'] = win[c].std()
            for c in categorical_cols:
                mode = win[c].mode()
                features[c] = mode.iloc[0] if not mode.empty else None

        rows.append(features)

    return pd.DataFrame(rows)

# Executa a função usando o DataFrame `dados` carregado acima
windows_df = sliding_windows(dados, window_s=1.0, overlap=0.5)
print('Janelas geradas:', windows_df.shape)
windows_df.head()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\julia\.vscode\PIBIC\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 2201, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\julia\.vscode\PIBIC\.venv\Lib\site-packages\IPython\core\ultratb.py", line 1193, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\julia\.vscode\PIBIC\.venv\Lib\site-packages\IPython\core\ultratb.py", line 1064, in structured_traceback
    return VerboseTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\julia\.vscode\PIBIC\.venv\Lib\site-packages\IPython\core\ultratb.py", line 872, in structured_traceback
    formatted_exceptions: list[list[str]] = self.format_exception_as_a_whole(
                                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\julia\.vscode\PIBIC\.venv